In [2]:
import pandas as pd
import numpy as np
import matplotlib as plt
import datetime
import os

In [26]:
pnc_path = r"C:\Users\willd\OneDrive\Desktop\Budget\statements\pnc_csvs"
chase_path = r"C:\Users\willd\OneDrive\Desktop\Budget\statements\chase_csvs"

In [31]:
# combine multiple pnc statements
def combine(path):
    df_list = []
    for item in os.listdir(path):
        full_path = os.path.join(path, item)
        df = pd.read_csv(full_path)
        df_list.append(df)
    combined = pd.concat(df_list)
    combined.columns.values[0] = "Date"
    no_dup = combined.drop_duplicates(subset=['Date', 'Description'], keep='first', inplace=False, ignore_index=True)
    return no_dup

# function to fill ammount column in the pnc dataframe 
def amount_fill(data):
    if '$' in str(data['Withdrawals']):
        amount = '-' + data['Withdrawals']
    else:
        amount = data['Deposits']
    return float(amount.replace('$', '').replace(',', ''))

In [32]:
pnc_full = combine(pnc_path)
chase_full = combine(chase_path)

In [33]:
pnc_full['Amount'] = pnc_full.apply(amount_fill, axis=1)

In [35]:
# add catacories to pnc data frame 
df = pnc_full['Description']
df2 = pnc_full['Amount']
cat_condition = [(df.str.contains('VENMO')) & (df2 > 0), (df.str.contains('VENMO')) & (df2 < 0), (df.str.contains('SPOTIFY') | df.str.contains('APPLE')),
                 (df.str.contains('ACTALENT')), (df.str.contains('CHECK'))
                , (df.str.contains('SCHWAB BROK'))]
cat_value = ['VENMO IN','VENMO OUT', 'SUBSCRIP', 'Belair', 'RENT', 'SCHWAB']
pnc_full['Category'] = np.select(cat_condition, cat_value)
# make a misc catagory of remaining  
pnc_full.loc[pnc_full['Category'] == '0', 'Category'] = 'Misc'

In [40]:
# reduce both data frames to relevant columns
chase_reduced = chase_full[["Date", 'Description', "Category", "Amount"]]
pnc_reduced = pnc_full[["Date", 'Description', "Category", "Amount"]]

# remove CC Payments from both 
pnc_no_cc = pnc_reduced[~pnc_reduced['Description'].str.contains('CHASE CREDIT CRD EPA')]
# remove pnc transfers 
pnc_no_transfer = pnc_no_cc[~pnc_no_cc['Description'].str.contains('ONLINE TRANSFER')]
chase_no_cc = chase_reduced[chase_reduced['Amount'] < 0]

# combine pnc and chase for analysis 
both = [chase_no_cc, pnc_no_cc]
full_df = pd.concat(both)

# change date dtype
full_df['Date'] = pd.to_datetime(full_df['Date'])

#reduce data set to only include same months 
full_df = full_df[full_df['Date'] >= '2023-09-01']

# add month column 
full_df['Month'] = pd.DatetimeIndex(full_df['Date']).month

In [41]:
full_df

,Date,Description,Category,Amount,Month
1,2023-11-13,SONIC DRIVE IN #4415,Food & Drink,-15.19,11
2,2023-11-10,BARRELS &amp; BOTTLES,Food & Drink,-71.56,11
3,2023-11-09,KING SOOPERS #0682 FUE,Gas,-55.20,11
4,2023-11-07,KING SOOPERS #0082,Groceries,-9.87,11
5,2023-11-07,MCDONALD'S F6010,Food & Drink,-12.44,11
...,...,...,...,...,...
54,2023-09-13,"ACH CREDIT 07987436 ACTALENT, INC. DIR DEP",Belair,1188.78,9
55,2023-09-11,POS PURCHASE POS10202026 5217430 Gre...,Misc,-41.43,9
56,2023-09-07,ACH DEBIT XXXXX7472 SCHWAB BROKERAGE MON...,SCHWAB,-200.00,9
58,2023-09-06,"ACH CREDIT 07987436 ACTALENT, INC. DIR DEP",Belair,1223.55,9


In [42]:
# remove venmo in > $500 from rent by mont 
month_list = full_df['Month'].unique()
# full_df.loc[pnc_full[''] == '0', 'Category'] = 'Misc'
#remove that venmo paymnet from money in 


In [43]:
month_list

array([11, 10,  9], dtype=int64)

In [15]:
money_in = full_df[full_df['Amount'] > 0]
money_out = full_df[full_df['Amount'] < 0]

In [16]:
money_out['Amount'] = 0 - money_out['Amount']

C:\Users\willd\AppData\Local\Temp\ipykernel_2744\2362525790.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  money_out['Amount'] = 0 - money_out['Amount']


In [22]:
money_in[money_in['Category'] == 'VENMO IN']

,Date,Description,Category,Amount,Month
13,2023-10-27,ACH CREDIT XXXXX5018 VENMO CAS...,VENMO IN,850.00,10
35,2023-09-27,ACH CREDIT XXXXX5291 VENMO CAS...,VENMO IN,909.11,9


In [17]:
expense_pivot = money_out.pivot_table(values='Amount', index='Month', columns='Category', aggfunc='sum')

In [100]:
expense_pivot

Category,Automotive,Bills & Utilities,Entertainment,Food & Drink,Gas,Groceries,Health & Wellness,Home,Misc,RENT,SCHWAB,SUBSCRIP,Shopping,Travel,VENMO OUT
Month,,,,,,,,,,,,,,,
9,NaN,89.39,929.0,601.67,122.96,289.91,NaN,NaN,41.43,NaN,450.0,12.86,182.05,5.45,860.75
10,122.52,161.87,NaN,168.28,127.90,128.29,NaN,23.82,146.78,1560.0,2000.0,12.86,457.45,523.85,899.92
11,NaN,NaN,NaN,99.19,62.07,9.87,179.7,NaN,NaN,1680.0,NaN,NaN,25.78,NaN,320.82


In [107]:
month_exp = money_out.groupby('Month').sum()
month_inc = money_in.groupby('Month').sum()

In [108]:
print(month_exp)
print(month_inc)

        Amount
Month         
9      3585.47
10     6333.54
11     2377.43
        Amount
Month         
9      5699.02
10     5650.98
11     3566.36


In [110]:
x = month_exp[0]
y1 = month_exp['Amount']
y2 = month_inc['Amount']
plt.plot(x, y1)

KeyError: 0

In [73]:
ytd = money_in.loc[money_in['Category'] == 'Belair', 'Amount'].sum()
othermoney = money_in.loc[money_in['Category'] != 'Belair', 'Amount'].sum()
as_of = money_in['Date'].max()
expense = money_out['Amount'].sum() - othermoney
income_report = f"""
As of {as_of}, you have made a total of ${ytd}
And you have spent ${expense}"""
print(income_report)


As of 2023-11-15 00:00:00, you have made a total of $13151.24
And you have spent $10531.319999999998
